## TP3 - Exercice 3 (Détection d’anomalie : implémentation, pratique et observations)  - 17/10/2025

### Objectif :



### Question 1 — Matrices de paramètres du modèle statistique  
*(Algorithme classique des densités de probabilité partielles)*

**Objectif :**  
Déterminer les paramètres statistiques de la base de données afin de modéliser la distribution de chaque variable d’entrée selon une loi normale indépendante.

---

#### 🔹 Données utilisées :
Chaque individu est décrit par quatre caractéristiques :
- x1 : aller au café  
- x2 : faire des courses  
- x3 : faire du sport  
- x4 : rester à la maison  

Les valeurs sont encodées entre 0 et 1 selon le niveau de préférence.

#### 🔹 Calculs effectués :
Pour chaque variable \( x_j \), on calcule :
$$
\mu_j = \frac{1}{m}\sum_{i=1}^{m} x_j^{(i)} , \qquad
\sigma_j^2 = \frac{1}{m}\sum_{i=1}^{m}(x_j^{(i)} - \mu_j)^2
$$

Ainsi :
- **μ** : vecteur des moyennes (centre de chaque variable)  
- **σ²** : vecteur des variances (dispersion de chaque variable)  
- **Σ** : matrice de covariance diagonale (sous hypothèse d’indépendance)


#### 🧠 Interprétation :
- Les moyennes représentent le comportement moyen des opérateurs.  
- Les variances montrent la variabilité de chaque habitude.  
- La matrice Σ est diagonale car on suppose que les variables sont **indépendantes** dans ce modèle.

In [3]:
import numpy as np
import pandas as pd

data = np.array([
    [1.0, 0.0, 0.0, 0.3],
    [0.3, 0.3, 0.75, 0.0],
    [0.0, 1.0, 0.0, 0.3],
    [0.75, 0.3, 0.0, 0.3],
    [0.0, 1.0, 0.0, 0.3],
    [0.0, 0.0, 1.0, 0.3]
])

columns = ['x1 (café)', 'x2 (courses)', 'x3 (sport)', 'x4 (maison)']
df = pd.DataFrame(data, columns=columns)
df.index = np.arange(1, len(df)+1)
df


,x1 (café),x2 (courses),x3 (sport),x4 (maison)
1,1.00,0.0,0.00,0.3
2,0.30,0.3,0.75,0.0
3,0.00,1.0,0.00,0.3
4,0.75,0.3,0.00,0.3
5,0.00,1.0,0.00,0.3
6,0.00,0.0,1.00,0.3


In [4]:

# -----------------------------
# Calcul des paramètres statistiques
# -----------------------------
mu = data.mean(axis=0)       # Moyennes
var = data.var(axis=0)       # Variances (population)
std = np.sqrt(var)           # Écarts-types

param_df = pd.DataFrame({
    "μ (moyenne)": mu.round(4),
    "σ² (variance)": var.round(6),
    "σ (écart-type)": std.round(4)
}, index=columns)

print("\n🔹 Matrices de paramètres statistiques :")
display(param_df)

# -----------------------------
# Matrice diagonale des variances Σ_diag
# (modèle indépendant)
# -----------------------------
Sigma_diag = np.diag(var)
print("\n🔹 Matrice diagonale Σ (sous indépendance) :")
display(pd.DataFrame(Sigma_diag, index=columns, columns=columns))


🔹 Matrices de paramètres statistiques :


,μ (moyenne),σ² (variance),σ (écart-type)
x1 (café),0.3417,0.158681,0.3983
x2 (courses),0.4333,0.175556,0.4190
x3 (sport),0.2917,0.175347,0.4187
x4 (maison),0.2500,0.012500,0.1118



🔹 Matrice diagonale Σ (sous indépendance) :


,x1 (café),x2 (courses),x3 (sport),x4 (maison)
x1 (café),0.158681,0.000000,0.000000,0.0000
x2 (courses),0.000000,0.175556,0.000000,0.0000
x3 (sport),0.000000,0.000000,0.175347,0.0000
x4 (maison),0.000000,0.000000,0.000000,0.0125


### Question 2 — Déterminer s’il existe une anomalie dans la base

**Principe (modèle “densités partielles”, variables indépendantes)**  
On suppose que chaque variable suit une loi normale indépendante :  
$x_j \sim \mathcal{N}(\mu_j, \sigma_j^2)$  

La probabilité d’un échantillon $x = (x_1, \dots, x_n)$ est le **produit** des densités univariées :

$$
p(x) = \prod_{j=1}^{n} \mathcal{N}(x_j; \mu_j, \sigma_j^2)
$$

---

**Règle de décision :**
- Si $p(x) \le \varepsilon$, alors l’observation est considérée comme une **anomalie**.  
- Si $p(x) > \varepsilon$, l’observation est **normale**.

Le seuil $\varepsilon$ peut être choisi de deux manières :
- **Fixe** : par exemple $\varepsilon = 0.02$  
- **Adaptatif** : 5e percentile des probabilités $p(x_i)$ dans la base d’entraînement.

---

**Étapes de l’analyse :**
1. Calculer les probabilités $p(x_i)$ de chaque individu.  
2. Fixer le seuil $\varepsilon$.  
3. Comparer chaque $p(x_i)$ à $\varepsilon$.  
4. Identifier les individus dont $p(x_i) \le \varepsilon$ comme **anormaux**.

---

**Interprétation :**  
Une probabilité $p(x)$ très faible indique que l’échantillon est **peu vraisemblable** selon le modèle appris.  
Ces points peuvent être considérés comme **anomalies** dans la base.


### 💡 Pourquoi on parle du cas univarié ?

Même si on a plusieurs variables ($x_1, x_2, x_3, x_4$),  
le modèle **classique** suppose qu’elles sont **indépendantes** les unes des autres.

On calcule donc la densité de chaque variable séparément :

$$
p(x) = p(x_1) \times p(x_2) \times p(x_3) \times p(x_4)
$$

👉 Chaque $p(x_j)$ suit une loi normale **univariée** (1 seule variable).  
C’est pourquoi on parle d’un **modèle univarié** ou de **densités partielles**.  

Le modèle **multivarié**, qu’on verra plus tard, tient compte des **corrélations** entre les variables.


In [12]:
import numpy as np
import pandas as pd
import math

# ------- Densité partielle (produit de gaussiennes univariées) -------
def p_independent(x, mu, var):
    p = 1.0
    for j in range(len(mu)):
        if var[j] <= 1e-12:  # sécurité si variance quasi nulle
            return 0.0
        coef = 1.0 / math.sqrt(2*math.pi*var[j])
        expo = math.exp(- (x[j]-mu[j])**2 / (2*var[j]))
        p *= coef * expo
    return p

# ------- Paramètres statistiques -------
mu  = data.mean(axis=0)
var = data.var(axis=0)

# ------- Calcul des probabilités -------
p_train = np.array([p_independent(x, mu, var) for x in data])
df_p = pd.DataFrame({'p(x_i)': p_train}, index=df.index)

# ------- Seuil fixe -------
eps_fixed = 0.02
df_p['anomalie (p<=0.02)'] = p_train <= eps_fixed

print("Seuil fixe (0.02) =", eps_fixed)
df_p


Seuil fixe (0.02) = 0.02


,p(x_i),anomalie (p<=0.02)
0,0.344058,False
1,0.138203,False
2,0.638342,False
3,1.293684,False
4,0.638342,False
5,0.284434,False


### Question 3 — Calcul de la matrice de covariance de la base \(X_B\)

**Objectif :**  
La matrice de covariance mesure comment les variables varient **ensemble** :
- Si deux variables augmentent ou diminuent ensemble → covariance **positive**
- Si l’une augmente quand l’autre diminue → covariance **négative**
- Si elles sont indépendantes → covariance ≈ **0**

---

**Formule générale :**

$$
\text{cov}(x_i, x_j) = \frac{1}{m-1} \sum_{k=1}^{m} (x_i^{(k)} - \mu_i)(x_j^{(k)} - \mu_j)
$$

où :
- \(m\) = nombre d’observations  
- \(\mu_i, \mu_j\) = moyennes des variables \(x_i\) et \(x_j\)

---

**Matrice de covariance complète :**

$$
\Sigma =
\begin{bmatrix}
\text{cov}(x_1, x_1) & \text{cov}(x_1, x_2) & \text{cov}(x_1, x_3) & \text{cov}(x_1, x_4) \\
\text{cov}(x_2, x_1) & \text{cov}(x_2, x_2) & \text{cov}(x_2, x_3) & \text{cov}(x_2, x_4) \\
\text{cov}(x_3, x_1) & \text{cov}(x_3, x_2) & \text{cov}(x_3, x_3) & \text{cov}(x_3, x_4) \\
\text{cov}(x_4, x_1) & \text{cov}(x_4, x_2) & \text{cov}(x_4, x_3) & \text{cov}(x_4, x_4)
\end{bmatrix}
$$

---

**Interprétation :**
- Les **valeurs diagonales** sont les **variances** des variables.  
- Les **valeurs hors diagonale** montrent les **relations entre variables** (covariances croisées).  
- Si la matrice est presque diagonale → les variables sont **faiblement corrélées**.


In [11]:
# ------- Calcul de la matrice de covariance -------
cov_matrix = np.cov(data, rowvar=False)  # rowvar=False → chaque colonne = une variable
cov_df = pd.DataFrame(cov_matrix, columns=columns, index=columns)

print("✅ Matrice de covariance Σ :")
cov_df


✅ Matrice de covariance Σ :


,x1 (café),x2 (courses),x3 (sport),x4 (maison)
x1 (café),0.190417,-0.114667,-0.074583,0.0025
x2 (courses),-0.114667,0.210667,-0.106667,0.0080
x3 (sport),-0.074583,-0.106667,0.210417,-0.0275
x4 (maison),0.002500,0.008000,-0.027500,0.0150


### Question 4 — Dépendances entre les caractéristiques d’entrée

**Objectif :**  
Identifier les relations entre les variables à partir de la matrice de covariance.

---

#### 🔹 Interprétation simple
- **Covariance ≈ 0** → variables indépendantes  
- **Covariance > 0** → varient dans le même sens  
- **Covariance < 0** → varient en sens inverse  

---

#### 🔹 Analyse des dépendances

| Variables | Covariance | Niveau | Interprétation |
|------------|-------------|----------|----------------|
| x1–x2 | -0.115 | Moyenne | Aller au café ↔ faire les courses (relation inverse) |
| x2–x3 | -0.107 | Moyenne | Courses ↔ sport (légère opposition) |
| x1–x3 | -0.075 | Faible | Café ↔ sport (relation faible) |
| Autres paires | ≈ 0 | Très faible | Variables presque indépendantes |

---

#### 🔹 Conclusion
> Les dépendances les plus marquées concernent **x1–x2** et **x2–x3**,  
> avec des **covariances négatives moyennes**, indiquant une **légère opposition** entre ces activités.  
> Globalement, les variables sont **faiblement corrélées**, ce qui justifie l’usage du **modèle univarié**.


### Question 6 — Détermination de l’acceptabilité des nouvelles valeurs

**Objectif :**
Tester si les 4 nouveaux individus peuvent être considérés comme "normaux" selon le modèle statistique appris.

On utilise le même modèle de détection d’anomalies (densités partielles indépendantes) basé sur :
- Les moyennes μ et variances σ² de la base d’origine (6 individus).
- Le seuil fixe choisi précédemment (ε = 0.02).

**Principe de décision :**
$$
\text{Si } p(x_{\text{new}}) \leq \varepsilon \Rightarrow \text{anomalie}
\quad \text{sinon} \Rightarrow \text{valeur normale.}
$$


In [22]:
# ====== Nouvelle base consolidée (10 personnes) ======
data_consolidee = np.array([
    [1.0, 0.0, 0.0, 0.3],
    [0.3, 0.3, 0.75, 0.0],
    [0.0, 1.0, 0.0, 0.3],
    [0.75, 0.3, 0.0, 0.3],
    [0.0, 1.0, 0.0, 0.3],
    [0.0, 0.0, 1.0, 0.3],
    [0.85, 0.15, 0.00, 0.00],
    [1.00, 0.00, 0.00, 0.00],
    [0.00, 0.15, 0.80, 0.60],
    [0.40, 0.15, 0.40, 0.40]
])

# ====== Calcul de la nouvelle moyenne et variance ======
mu_new = data_consolidee.mean(axis=0)
var_new = data_consolidee.var(axis=0)

print("Nouvelle moyenne :", mu_new)
print("Nouvelle variance :", var_new)

# ====== Calcul des probabilités avec les nouveaux paramètres ======
p_new = np.array([p_independent(x, mu_new, var_new) for x in data_consolidee])
eps_fixed = 0.02

df_new = pd.DataFrame(data_consolidee, columns=['x1 (café)', 'x2 (courses)', 'x3 (sport)', 'x4 (maison)'])
df_new['p(x_new)'] = p_new
df_new['anomalie (p<=0.02)'] = p_new <= eps_fixed

df_new


Nouvelle moyenne : [0.43  0.305 0.295 0.25 ]
Nouvelle variance : [0.1686   0.131725 0.149225 0.0345  ]


,x1 (café),x2 (courses),x3 (sport),x4 (maison),p(x_new),anomalie (p<=0.02)
0,1.00,0.00,0.00,0.3,0.457480,False
1,0.30,0.30,0.75,0.0,0.455094,False
2,0.00,1.00,0.00,0.3,0.157679,False
3,0.75,0.30,0.00,0.3,1.259648,False
4,0.00,1.00,0.00,0.3,0.157679,False
5,0.00,0.00,1.00,0.3,0.175415,False
6,0.85,0.15,0.00,0.0,0.387017,False
7,1.00,0.00,0.00,0.0,0.191745,False
8,0.00,0.15,0.80,0.6,0.090088,False
9,0.40,0.15,0.40,0.4,1.500106,False


### Résultats de l’évaluation sur la base consolidée

L’analyse des probabilités montre que tous les individus possèdent des valeurs
de densité \( p(x_i) > 0.02 \). Cela signifie qu’aucun profil n’est considéré
comme une anomalie statistique.

Les nouvelles personnes présentent des comportements cohérents avec la base initiale.
Elles peuvent donc être intégrées définitivement dans la base consolidée sans
perturber la distribution générale.

👉 En conclusion, la base finale (10 individus) est homogène et statistiquement stable.


### Interprétation

Toutes les nouvelles valeurs sont **acceptables pour insertion** dans la base.
Elles peuvent donc être intégrées pour constituer une **base consolidée**
représentative du comportement global des individus.


### Question 7 — Les nouveaux exemples appartiennent-ils aux groupements déjà fixés ?

**Idée générale :**  
On **ne ré-entraîne pas** K-Means. On garde les **centroïdes finaux** déjà fixés (Ex. 1) et on vérifie si chaque nouveau point est **cohérent** avec le cluster le plus proche.

#### Règle d’évaluation 

1. **Distance au centroïde (rayon intra-cluster)**  
   - Pour chaque cluster $k$, on calcule sur les **données d’entraînement** son **rayon** $r_k$ = *max* distance d’un point du cluster à son centroïde.  
   - Un nouveau point affecté au cluster $k$ est **accepté** si  
     $$
     \text{dist}(x, G_k) \le r_k
     $$

> **Décision :** accepter si **(rayon OK)**.  
> (Règle « OR » = prudente ; pour être plus strict, utiliser « AND ».)

**Pourquoi c’est pertinent ?**  
- Le **rayon** contrôle l’**ampleur** du cluster (distance brute).  


In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# ====== 1️⃣ Base initiale (6 personnes) ======
X_train = np.array([
    [1.0, 0.0, 0.0, 0.3],
    [0.3, 0.3, 0.75, 0.0],
    [0.0, 1.0, 0.0, 0.3],
    [0.75, 0.3, 0.0, 0.3],
    [0.0, 1.0, 0.0, 0.3],
    [0.0, 0.0, 1.0, 0.3]
])

# ====== 2️⃣ Nouvelles personnes ======
X_new = np.array([
    [0.85, 0.15, 0.00, 0.00],
    [1.00, 0.00, 0.00, 0.00],
    [0.00, 0.15, 0.80, 0.60],
    [0.40, 0.15, 0.40, 0.40]
])

# ====== 3️⃣ Appliquer K-Means sur la base initiale ======
kmeans = KMeans(n_clusters=2, random_state=0, n_init=10)
kmeans.fit(X_train)

centers = kmeans.cluster_centers_
labels = kmeans.labels_

# ====== 4️⃣ Calcul des rayons intra-cluster ======
def euclid(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

radii = []
for k in range(2):
    cluster_points = X_train[labels == k]
    distances = [euclid(x, centers[k]) for x in cluster_points]
    rayon_k = np.max(distances)
    radii.append(rayon_k)

print("Centroïdes des clusters :\n", centers)
print("\nRayons des clusters :", radii)

# ====== 5️⃣ Évaluation des nouvelles personnes (euclid seulement) ======
rows = []
for x in X_new:
    # Distance euclidienne au centroïde le plus proche
    dists = [euclid(x, c) for c in centers]
    k = int(np.argmin(dists))
    d_to_center = dists[k]
    within_radius = d_to_center <= radii[k]

    # Décision finale basée uniquement sur le rayon
    accepted = within_radius

    rows.append({
        "x1(café)": x[0],
        "x2(courses)": x[1],
        "x3(sport)": x[2],
        "x4(maison)": x[3],
        "cluster_attribué": k + 1,
        "dist_centroïde": round(d_to_center, 3),
        "rayon_cluster": round(radii[k], 3),
        "within_radius": within_radius,
        "ACCEPTE": accepted
    })

# ====== 6️⃣ Résultat final ======
df_result = pd.DataFrame(rows)
print("\nRésultats finaux :")
df_result


Centroïdes des clusters :
 [[0.5125 0.15   0.4375 0.225 ]
 [0.     1.     0.     0.3   ]]

Rayons des clusters : [0.7792223687754349, 0.0]

Résultats finaux :


,x1(café),x2(courses),x3(sport),x4(maison),cluster_attribué,dist_centroïde,rayon_cluster,within_radius,ACCEPTE
0,0.85,0.15,0.0,0.0,1,0.597,0.779,True,True
1,1.00,0.00,0.0,0.0,1,0.709,0.779,True,True
2,0.00,0.15,0.8,0.6,1,0.731,0.779,True,True
3,0.40,0.15,0.4,0.4,1,0.211,0.779,True,True


### Interprétation

- Toutes les nouvelles personnes sont acceptées dans le cluster 1, car leur distance euclidienne par rapport au centroïde de ce cluster est inférieure au rayon du cluster (0.779).
- Le cluster 2 n’a aucun nouveau point assigné et a un rayon de 0, ce qui montre qu’il ne contient probablement qu’un seul point dans l’échantillon initial.
